# 1. Preprocess-GroupImages

数据预处理：主要是将图片数据解压，然后复制到ImageDataGenerator需要的文件目录中。

## Import PKGs

In [1]:
import time
import os
import json
import csv
import zipfile
import pickle
from PIL import Image
import shutil


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from IPython.display import display

## Run name

In [2]:
project_name = 'ic_furniture2018'
step_name = 'Preprocess-GroupImages'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: ic_furniture2018_Preprocess-GroupImages_20180329_114847


## Project folders

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('post_pca_feature_folder: \t%s' % post_pca_feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_val_folder = os.path.join(input_folder, 'org_val')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

train_json_file = os.path.join(input_folder, 'train.json')
val_json_file = os.path.join(input_folder, 'validation.json')
test_json_file = os.path.join(input_folder, 'test.json')
print('\ntrain_json_file: \t\t%s' % train_json_file)
print('val_json_file: \t\t%s' % val_json_file)
print('test_json_file: \t\t%s' % test_json_file)

train_csv_file = os.path.join(input_folder, 'train.csv')
val_csv_file = os.path.join(input_folder, 'validation.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('val_csv_file: \t\t%s' % val_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)

sample_submission_csv_file = os.path.join(input_folder, 'sample_submission_randomlabel.csv')
print('\nsample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input
output_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/output
model_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/model
feature_folder: 		/data1/kaggle/imaterialist-challenge-furniture-2018/feature
post_pca_feature_folder: 	/data1/kaggle/imaterialist-challenge-furniture-2018/post_pca_feature
log_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/log

train_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.json
val_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.json
test_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.json

train_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.csv
val_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.csv
test_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.csv

s

## Preview data

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

val_csv = pd.read_csv(val_csv_file)
print('train_csv.shape is {0}.'.format(val_csv.shape))
display(val_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

train_csv.shape is (194828, 3).


,image_id,label_id,url
0,1,5,https://img13.360buyimg.com/imgzone/jfs/t2857/...
1,2,5,http://www.tengdakeli.cn/350/timg01/uploaded/i...


train_csv.shape is (6400, 3).


,image_id,label_id,url
0,1,38,http://www.ghs.net/public/images/fb/3d/51/3beb...
1,2,63,https://img.alicdn.com/imgextra/TB2chFei9YH8KJ...


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


sample_submission_csv.shape is (12800, 2).


,id,predicted
0,1,57
1,2,74


In [5]:
train_id = train_csv['image_id']
train_label_id = train_csv['label_id']

id_2_train_label_id_dict = dict(zip(train_id, train_label_id))
print('len(id_2_train_label_id_dict)=%d' % len(id_2_train_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))

image_file = '%s_%s.jpg' % (train_id[index], id_2_train_label_id_dict[train_id[index]])
print(image_file)

len(id_2_train_label_id_dict)=194828
id: 1, 	landmark_id:5
id: 2, 	landmark_id:5
2_5.jpg


In [6]:
val_id = val_csv['image_id']
val_label_id = val_csv['label_id']

id_2_val_label_id_dict = dict(zip(val_id, val_label_id))
print('len(id_2_val_label_id_dict)=%d' % len(id_2_val_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))

image_file = '%s_%s.jpg' % (val_id[index], id_2_val_label_id_dict[val_id[index]])
print(image_file)

len(id_2_val_label_id_dict)=6400
id: 1, 	landmark_id:38
id: 2, 	landmark_id:63
2_63.jpg


In [7]:
test_id = test_csv['image_id']

index = 0
print('id: %s' % (test_id[index]))
index = 1
print('id: %s' % (test_id[index]))

image_file = '%s.jpg' % (test_id[index])
print(image_file)

id: 1
id: 2
2.jpg


## Create data_train，data_val, data_test folders and their subfolders

In [8]:
if not os.path.exists(train_folder):
    print('Create folder: %s' % train_folder)
    os.mkdir(train_folder)
else:
    print('Folder exists: %s' % train_folder)

if not os.path.exists(val_folder):
    print('Create folder: %s' % val_folder)
    os.mkdir(val_folder)
else:
    print('Folder exists: %s' % train_folder)

    
if not os.path.exists(test_folder):
    print('Create folder: %s' % test_folder)
    os.mkdir(test_folder)
else:
    print('Folder exists: %s' % test_folder)

if not os.path.exists(test_sub_folder):
    print('Create folder: %s' % test_sub_folder)
    os.mkdir(test_sub_folder)
else:
    print('Folder exists: %s' % test_sub_folder)

Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_test
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_test/test


In [9]:
unique_train_label_ids = list(set(train_label_id))
print('len(unique_train_label_ids)=%d' % len(unique_train_label_ids))
sub_folder_name_len = len(str(len(unique_train_label_ids)))
print('sub_folder_name_len=%d' % sub_folder_name_len)

len(unique_train_label_ids)=128
sub_folder_name_len=3


In [10]:
for c in unique_train_label_ids:
    train_sub_folder = os.path.join(train_folder, str(c).zfill(sub_folder_name_len))
    if not os.path.exists(train_sub_folder):
        print('Create folder: %s' % train_sub_folder)
        os.mkdir(train_sub_folder)
    else:
        print('Folder exists: %s' % train_sub_folder)

Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/001
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/002
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/003
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/004
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/005
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/006
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/007
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/008
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/009
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/010
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_train/011
Folder exists: /data1/kaggle/ima

In [11]:
for c in unique_train_label_ids:
    val_sub_folder = os.path.join(val_folder, str(c).zfill(sub_folder_name_len))
    if not os.path.exists(val_sub_folder):
        print('Create folder: %s' % val_sub_folder)
        os.mkdir(val_sub_folder)
    else:
        print('Folder exists: %s' % val_sub_folder)

Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/001
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/002
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/003
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/004
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/005
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/006
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/007
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/008
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/009
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/010
Folder exists: /data1/kaggle/imaterialist-challenge-furniture-2018/input/data_val/011
Folder exists: /data1/kaggle/imaterialist-challenge-fu

## Move images to corresponding subfolders

In [12]:
%%time
i = 0
data_train_amount = len(train_label_id)
data_train_images = os.listdir(org_train_folder)
len_data_train_images = len(data_train_images)
print(data_train_amount, len_data_train_images)

for image_name in data_train_images:
    image_info = image_name.replace('.', '_')
    image_info = image_info.split('_')
    src_img = os.path.join(org_train_folder, image_name)
    target_folder = os.path.join(train_folder, str(id_2_train_label_id_dict[int(image_info[0])]).zfill(sub_folder_name_len))
#     print(image_info)
#     print(src_img)
#     print(target_folder)
    shutil.copy(src_img, target_folder)
    i = i+1
    if i%1000 == 0:
        print('%.2f' % (i/len_data_train_images), end='  ')

194828 191261
0.01  0.01  0.02  0.02  0.03  0.03  0.04  0.04  0.05  0.05  0.06  0.06  0.07  0.07  0.08  0.08  0.09  0.09  0.10  0.10  0.11  0.12  0.12  0.13  0.13  0.14  0.14  0.15  0.15  0.16  0.16  0.17  0.17  0.18  0.18  0.19  0.19  0.20  0.20  0.21  0.21  0.22  0.22  0.23  0.24  0.24  0.25  0.25  0.26  0.26  0.27  0.27  0.28  0.28  0.29  0.29  0.30  0.30  0.31  0.31  0.32  0.32  0.33  0.33  0.34  0.35  0.35  0.36  0.36  0.37  0.37  0.38  0.38  0.39  0.39  0.40  0.40  0.41  0.41  0.42  0.42  0.43  0.43  0.44  0.44  0.45  0.45  0.46  0.47  0.47  0.48  0.48  0.49  0.49  0.50  0.50  0.51  0.51  0.52  0.52  0.53  0.53  0.54  0.54  0.55  0.55  0.56  0.56  0.57  0.58  0.58  0.59  0.59  0.60  0.60  0.61  0.61  0.62  0.62  0.63  0.63  0.64  0.64  0.65  0.65  0.66  0.66  0.67  0.67  0.68  0.68  0.69  0.70  0.70  0.71  0.71  0.72  0.72  0.73  0.73  0.74  0.74  0.75  0.75  0.76  0.76  0.77  0.77  0.78  0.78  0.79  0.79  0.80  0.81  0.81  0.82  0.82  0.83  0.83  0.84  0.84  0.85  0.85  0.86  0.

In [13]:
%%time
i = 0
data_val_amount = len(val_label_id)
data_val_images = os.listdir(org_val_folder)
len_data_val_images = len(data_val_images)
print(data_val_amount, len_data_val_images)

for image_name in data_val_images:
    image_info = image_name.replace('.', '_')
    image_info = image_info.split('_')
    src_img = os.path.join(org_val_folder, image_name)
    target_folder = os.path.join(val_folder, str(id_2_val_label_id_dict[int(image_info[0])]).zfill(sub_folder_name_len))
#     print(image_info)
#     print(src_img)
#     print(target_folder)
    shutil.copy(src_img, target_folder)
    i = i+1
    if i%1000 == 0:
        print('%.2f' % (i/len_data_val_images), end='  ')

6400 6301
0.16  0.32  0.48  0.63  0.79  0.95  CPU times: user 1.05 s, sys: 1.62 s, total: 2.68 s
Wall time: 1min 22s


In [14]:
%%time
i = 0
data_test_amount = len(test_csv['image_id'])
data_test_images = os.listdir(org_test_folder)
len_data_test_images = len(data_test_images)
print(data_test_amount, len_data_test_images)

for image_name in data_test_images:
    src_img = os.path.join(org_test_folder, image_name)
    target_folder = test_sub_folder
#     print(image_name)
#     print(src_img)
#     print(target_folder)
    shutil.copy(src_img, target_folder)
    i = i+1
    if i%1000 == 0:
        print('%.2f' % (i/len_data_test_images), end='  ')

12800 12652
0.08  0.16  0.24  0.32  0.40  0.47  0.55  0.63  0.71  0.79  0.87  0.95  CPU times: user 1.44 s, sys: 3.58 s, total: 5.01 s
Wall time: 2min 48s


In [15]:
print('Done!')

Done!
